# Test Vector Leakage Assessment

An example.

### 1. Generating examples

In this example, there are two keys $k_a$ and $k_b$. For each of these 2 keys, there are $1000$ power traces (samples)
available, each with a length of $10$ sample points.

In [30]:
from src.tvla.t import make_t_test
from src.tvla.main import tvla
samples = 1000
sample_pts = 1400

The samples for these imaginary traces will be simplified into normal distributions with mean $\mu$.

A first-order "leakage" will be simulated by taking $\mu_a$ (for key $k_a$) slightly bigger than $\mu_b$.

In [31]:
mu_a = 42.2
mu_b = 42

In [32]:
import numpy as np

# Refer to section statistical properties of ASCAD in books/data/ascad_normality.
sigma = .2

traces_key_a = np.array([np.random.normal(size=samples, scale=sigma) + mu_a for _ in range(sample_pts)], dtype=int)
traces_key_b = np.array([np.random.normal(size=samples, scale=sigma) + mu_b for _ in range(sample_pts)], dtype=int)

### 2. Slicing the input

The first step in TVLA is to split these samples into four disjoint slices, each with a length of $250$.

The $n$-th slice for key $k_a$ will be denoted as $A_n$.

In [33]:
def make_slice(traces: np.array):
    select = np.array(range(traces.shape[1]))
    np.random.shuffle(select)

    return np.array([traces[:, s] for s in np.array_split(select, 4)])

A = make_slice(traces_key_a)
B = make_slice(traces_key_b)

In [34]:
A.shape

(4, 1400, 250)

### 3. Preparing the data

In this example, the t-test is used when performing TVLA. The t-test takes the sample mean, sample variance and some
value for $p$. It tests the null hypothesis that 2 independent samples have identical average (expected) values, and
returns whether the null hypotheses should be accepted.

Therefore, $A$ and $B$ need to be transformed to fit the input parameters of the t-test.

In [35]:
A_mv = np.moveaxis(np.array((A.mean(axis=2), A.var(axis=2))), 0, -1)
B_mv = np.moveaxis(np.array((B.mean(axis=2), B.var(axis=2))), 0, -1)

A_mv.shape

(4, 1400, 2)

### 4. Applying TVLA

In [36]:
num_observations = A.shape[2]
t_test = make_t_test(num_observations)

tvla(test=t_test, left=A_mv, right=B_mv, p=5)

(True, False)